# データ取得

## インポート・設定

In [50]:
from pathlib import Path
import urllib.request
import zipfile
from pprint import pprint

import estatapi
from dotenv import dotenv_values

In [33]:
APPID = dotenv_values()["ESTAT_APPID"]
estatapi.set_appid(APPID)

DATA_DIR = Path("./data")

## データ取得

### 人口データ

[人口推計 \| 政府統計の総合窓口](https://www.e-stat.go.jp/statistics/00200524)

[人口推計 各年10月1日現在人口 令和２年国勢調査基準 統計表005 都道府県，男女別人口－総人口，日本人人口 \| 統計表・グラフ表示 \| 政府統計の総合窓口](https://www.e-stat.go.jp/dbview?sid=0003448232)

In [29]:
statsDataId = "0003448232"

データを取得する

In [30]:
stats_data_response = estatapi.get_stats_data(statsDataId=statsDataId)
df_stats_data = estatapi.stats_data_to_pandas(stats_data_response.json())
df_stats_data.head()

,表章項目,表章項目_階層,男女別,男女別_階層,人口,人口_階層,全国・都道府県,全国・都道府県_階層,時間軸（年）,時間軸（年）_階層,単位,値
0,人口,,男女計,1,総人口,1,全国,1,2005年,1,千人,127768
1,人口,,男女計,1,総人口,1,全国,1,2010年,1,千人,128057
2,人口,,男女計,1,総人口,1,全国,1,2015年,1,千人,127095
3,人口,,男女計,1,総人口,1,全国,1,2020年,1,千人,126146
4,人口,,男女計,1,総人口,1,全国,1,2021年,1,千人,125502


csvで保存する

In [34]:
df_stats_data.to_csv(DATA_DIR / "population.csv", index=False)

### 行政区画データ

行政区画のデータを取得する

[国土数値情報 \| 行政区域データ](https://nlftp.mlit.go.jp/ksj/gml/datalist/KsjTmplt-N03-v3_1.html)

In [47]:
url = "https://nlftp.mlit.go.jp/ksj/gml/data/N03/N03-2023/N03-20230101_GML.zip"
download_path = DATA_DIR / Path(url).name
urllib.request.urlretrieve(url=url, filename=download_path)

zipファイルの中身を確認

In [52]:
with zipfile.ZipFile(download_path) as zf:
    namelist = zf.namelist()
pprint(namelist)

['KS-META-N03-23_230101.xml',
 'N03-23_230101.dbf',
 'N03-23_230101.geojson',
 'N03-23_230101.prj',
 'N03-23_230101.shp',
 'N03-23_230101.shx',
 'N03-23_230101.xml']


geojsonファイルを取り出す

In [54]:
geojson_filename = [name for name in namelist if name.endswith(".geojson")][0]
geojson_filename

'N03-23_230101.geojson'

In [55]:
with zipfile.ZipFile(download_path) as zf:
    zf.extract(geojson_filename, DATA_DIR)